In [1]:
!pip install tensorflow_model_optimization

import numpy as np
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, datasets, utils, callbacks, optimizers, losses
from keras.datasets import cifar10
import tensorflow_model_optimization as tfmot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 5.0 MB/s eta 0:00:00


In [2]:
# Loading data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Data Transform
x_train = x_train.astype(np.float32) / 255.0
y_train = utils.to_categorical(y_train)
x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean

x_test = x_test.astype(np.float32) / 255.0
y_test = utils.to_categorical(y_test)
x_test -= x_train_mean

170498071/170498071 [==============================] - 13s 0us/step


### ResNet18

In [3]:
kernel_initializer = 'he_normal'
input_shape = (32,32,3)

image_input = layers.Input(shape=input_shape)
# first layer

block1 = keras.Sequential([
    layers.Conv2D(kernel_size=3, filters=64, strides=1, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization(),
    layers.ReLU()
])(image_input)

# first stage
shortcut = block1
block2 = keras.Sequential([
    layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization()
])(block1)

block2 = layers.add([block2, shortcut])
block2 = layers.ReLU()(block2)

shortcut = block2
block3 = keras.Sequential([
    layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization()
])(block2)

block3 = layers.add([block3, shortcut])
block3 = layers.ReLU()(block3)

# second stage
shortcut = keras.Sequential([layers.Conv2D(filters=128, kernel_size=1, strides=2),
                             layers.BatchNormalization()])(block3)
block4 = keras.Sequential([
    layers.Conv2D(filters=128, kernel_size=3, strides=2, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization()
])(block3)

block4 = layers.add([block4, shortcut])
block4 = layers.ReLU()(block4)

shortcut = block4
block5 = keras.Sequential([
    layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization()
])(block4)

block5 = layers.add([block5, shortcut])
block5 = layers.ReLU()(block5)

# third stage
shortcut = keras.Sequential([layers.Conv2D(filters=256, kernel_size=1, strides=2),
                             layers.BatchNormalization()])(block5)
block6 = keras.Sequential([
    layers.Conv2D(filters=256, kernel_size=3, strides=2, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization()
])(block5)

block6 = layers.add([block6, shortcut])
block6 = layers.ReLU()(block6)

shortcut = block6
block7 = keras.Sequential([
    layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization()
])(block6)

block7 = layers.add([block7, shortcut])
block7 = layers.ReLU()(block7)

#fourth stage
shortcut = keras.Sequential([layers.Conv2D(filters=512, kernel_size=1, strides=2),
                             layers.BatchNormalization()])(block7)
block8 = keras.Sequential([
    layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=512, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization()
])(block7)

block8 = layers.add([block8, shortcut])
block8 = layers.ReLU()(block8)

shortcut = block8
block9 = keras.Sequential([
    layers.Conv2D(filters=512, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=512, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
    layers.BatchNormalization()
])(block8)

block9 = layers.add([block9, shortcut])
block9 = layers.ReLU()(block9)

predictions = keras.Sequential([
    layers.GlobalAveragePooling2D(),
    layers.Flatten(),
    layers.Dense(10)
])(block9)

model = keras.Model(inputs=image_input, outputs=predictions)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(x_train,
          y_train,
          epochs=200,
          validation_split=0.1)

model.summary()

Epoch 1/200
1407/1407 [==============================] - 70s 42ms/step - loss: 0.0268 - accuracy: 0.9907 - val_loss: 0.8490 - val_accuracy: 0.8448
Epoch 2/200
1407/1407 [==============================] - 59s 42ms/step - loss: 0.0269 - accuracy: 0.9908 - val_loss: 0.8873 - val_accuracy: 0.8382
Epoch 3/200
1407/1407 [==============================] - 59s 42ms/step - loss: 0.0222 - accuracy: 0.9926 - val_loss: 1.0363 - val_accuracy: 0.8256
Epoch 4/200
1407/1407 [==============================] - 59s 42ms/step - loss: 0.0214 - accuracy: 0.9926 - val_loss: 0.9466 - val_accuracy: 0.8302
Epoch 5/200
1407/1407 [==============================] - 59s 42ms/step - loss: 0.0235 - accuracy: 0.9920 - val_loss: 0.9133 - val_accuracy: 0.8370
Epoch 6/200
1407/1407 [==============================] - 60s 42ms/step - loss: 0.0191 - accuracy: 0.9939 - val_loss: 0.9458 - val_accuracy: 0.8410
Epoch 7/200
1407/1407 [==============================] - 59s 42ms/step - loss: 0.0195 - accuracy: 0.9936 - val_loss: 0